# Run SKLearnProcessor: S3 CSV → Feature Engineering → Feature Store (Offline) + Train/Val/Test splits

Notebook นี้รันสคริปต์ `preprocess-scikit-retail-feature-store_NEW.py` ที่ทำงานดังนี้:

1. อ่าน raw CSV จาก S3 (ProcessingInput → `/opt/ml/processing/input/data/`)
2. ทำ feature engineering (calendar + promo features) และสร้าง `high_demand`
3. split เป็น `train / validation / test` แล้ว export เป็น CSV
4. ingest ทั้งหมดเข้า **SageMaker Feature Store (Offline store)**
5. รอให้ Offline store **Active** และรอให้ **Glue Data Catalog table** ปรากฏ (เพื่อ query ใน Athena ได้)

> ✅ หมายเหตุ: สคริปต์เวอร์ชันนี้ *ไม่* รัน Athena DDL เอง (หลีกเลี่ยงปัญหา SQL dialect ของ Athena engine) แต่จะรอให้ Feature Store สร้าง Glue table ให้อัตโนมัติ

In [1]:
import os
import boto3
import sagemaker
from time import gmtime, strftime

from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

print('Region:', region)
print('Bucket:', bucket)
print('Role:', role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Region: us-east-1
Bucket: sagemaker-us-east-1-423623839320
Role: arn:aws:iam::423623839320:role/service-role/SageMaker-ExecutionRole-20250705T232334


In [8]:
# -------------------------------
# 1) Raw data source (S3)
# -------------------------------
# 👉 ต้องเป็น S3 prefix/folder ที่มีไฟล์ .csv อยู่ข้างใน (1 ไฟล์หรือหลายไฟล์ก็ได้)
# ตัวอย่าง:
# raw_input_data_s3_uri = f"s3://{bucket}/retail-demand/raw/2025-12-07/"
raw_input_data_s3_uri = f"s3://{bucket}/retail-demand-forecasting/csv/"

# -------------------------------
# 2) Processing output prefix (timestamped เพื่อไม่ overwrite)
# -------------------------------
timestamp = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
preprocess_output_s3_uri = f"s3://{bucket}/retail-demand/preprocess-{timestamp}/"

# -------------------------------
# 3) Split parameters
# -------------------------------
train_split_percentage = 0.90
validation_split_percentage = 0.05
test_split_percentage = 0.05

# -------------------------------
# 4) Feature Store config
# -------------------------------
# ✅ แนะนำให้ FIXED name/prefix (เพื่อ append/upsert ข้อมูลรายวันเข้ากลุ่มเดิม)
feature_group_name = 'retail_product_demand_feature_group'
feature_store_offline_prefix = f"s3://{bucket}/feature-store/retail-demand/offline-store"

print('raw_input_data_s3_uri        =', raw_input_data_s3_uri)
print('preprocess_output_s3_uri     =', preprocess_output_s3_uri)
print('feature_group_name           =', feature_group_name)
print('feature_store_offline_prefix =', feature_store_offline_prefix)

raw_input_data_s3_uri        = s3://sagemaker-us-east-1-423623839320/retail-demand-forecasting/csv/
preprocess_output_s3_uri     = s3://sagemaker-us-east-1-423623839320/retail-demand/preprocess-2025-12-12-14-42-43/
feature_group_name           = retail_product_demand_feature_group
feature_store_offline_prefix = s3://sagemaker-us-east-1-423623839320/feature-store/retail-demand/offline-store


In [9]:
athena_database = "retail_demand"        # <-- change to your DB name
athena_table = "demand_product"        # <-- change to your table name

# Simple default: train on all rows
athena_query = f"SELECT * FROM {athena_database}.{athena_table}"

print("Athena database:", athena_database)
print("Athena table   :", athena_table)
print("Athena query   :", athena_query)


Athena database: retail_demand
Athena table   : demand_product
Athena query   : SELECT * FROM retail_demand.demand_product


## SKLearnProcessor

เราจะรันสคริปต์ preprocess ใน SageMaker Processing job โดย:
- input: `raw_input_data_s3_uri` → mount ไปที่ `/opt/ml/processing/input/data/`
- output: train/validation/test → upload ไปที่ `preprocess_output_s3_uri`

In [10]:
processing_instance_type = 'ml.c5.2xlarge'
processing_instance_count = 1

processor = SKLearnProcessor(
    framework_version='0.23-1',   # ใช้รุ่นเดียวกับที่เคยใช้ใน processing step ก่อนหน้า (Py3.7)
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    env={'AWS_DEFAULT_REGION': region},
    max_runtime_in_seconds=7200,
)

print('Processor image:', processor.image_uri)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


Processor image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3


In [11]:
# ตรวจสอบว่าไฟล์สคริปต์อยู่ใน working directory ของ notebook
script_path = 'preprocess-scikit-retail-feature-store.py'
print('Script exists?', os.path.exists(script_path), '->', script_path)
if not os.path.exists(script_path):
    raise FileNotFoundError(
        'Cannot find preprocess-scikit-retail-feature-store.py in the current directory. '        'Please place the script next to this notebook (same folder) before running.'
    )

Script exists? True -> preprocess-scikit-retail-feature-store.py


In [12]:
# job_name = f"retail-feature-store-{timestamp}".lower()
# job_name = job_name.replace('_', '-').replace(':', '-')
# # SageMaker job name length limit ~63 chars
# job_name = job_name[:63]

# print('Processing job name:', job_name)

In [14]:


processor.run(
    code='preprocess-scikit-retail-feature-store.py',
    inputs=[
        # ProcessingInput(
        #     source=raw_input_data_s3_uri,
        #     destination='/opt/ml/processing/input/data',
        # )
    ],
    outputs=[
        ProcessingOutput(
            output_name='train',
            source='/opt/ml/processing/output/retail_product/train',
            destination=preprocess_output_s3_uri + 'train/',
        ),
        ProcessingOutput(
            output_name='validation',
            source='/opt/ml/processing/output/retail_product/validation',
            destination=preprocess_output_s3_uri + 'validation/',
        ),
        ProcessingOutput(
            output_name='test',
            source='/opt/ml/processing/output/retail_product/test',
            destination=preprocess_output_s3_uri + 'test/',
        ),
    ],
    arguments=[
        '--input-data', '/opt/ml/processing/input/data/',
        '--output-data', '/opt/ml/processing/output/retail_product',
        '--train-split-percentage', str(train_split_percentage),
        '--validation-split-percentage', str(validation_split_percentage),
        '--test-split-percentage', str(test_split_percentage),
        '--feature-group-name', feature_group_name,
        '--feature-store-offline-prefix', feature_store_offline_prefix,
        # 👇 ใหม่: ส่ง Athena parameters เข้าไป
        "--athena-database", athena_database,
        "--athena-table", athena_table,
        "--athena-query", athena_query,

    ],
    # job_name=job_name,
    logs=True,
    wait=True
)

INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2025-12-12-14-43-13-900


.........Collecting sagemaker==2.24.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.4/397.4 kB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 8.1 MB/s eta 0:00:00
  Created wheel for sagemaker: filename=sagemaker-2.24.1-py2.py3-none-any.whl size=560557 sha256=75cb7b37ae3154c56ab8f68092440cbf5a868d90d86cc17d5286da58c534b84e
  Stored in directory: /root/.cache/pip/wheels/2b/15/53/af138b6475377b5d7671738e347161d0b654b75e708ab70f77
  Created wheel for protobuf3-to-dict: filename=protobuf3_to_dict-0.1.5-py3-none-any.whl size=4015 sha256=362331a0d8131abf1b591bc6fb722328bd6d39926cbca8ffabc5b84c7ccb9

In [ ]:
# ดึงชื่อ Processing Job
processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']
print('Processing job name:', processing_job_name)

running_job = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=processing_job_name,
    sagemaker_session=sess,
)

desc = running_job.describe()
outputs = {o['OutputName']: o['S3Output']['S3Uri'] for o in desc['ProcessingOutputConfig']['Outputs']}

processed_train_data_s3_uri = outputs.get('train')
processed_validation_data_s3_uri = outputs.get('validation')
processed_test_data_s3_uri = outputs.get('test')

print('Processed train S3:      ', processed_train_data_s3_uri)
print('Processed validation S3: ', processed_validation_data_s3_uri)
print('Processed test S3:       ', processed_test_data_s3_uri)

# เก็บไว้ใช้ต่อใน notebook train/evaluate/pipeline
%store processed_train_data_s3_uri
%store processed_validation_data_s3_uri
%store processed_test_data_s3_uri
%store feature_group_name
%store feature_store_offline_prefix

In [ ]:
# ดาวน์โหลด train split มาดู
!rm -rf data
!mkdir -p data/train
!aws s3 cp $processed_train_data_s3_uri data/train/ --recursive
!ls -al data/train | head

import glob
import pandas as pd

train_files = sorted(glob.glob('data/train/*.csv'))
print('Train files:', train_files)

df = pd.read_csv(train_files[0])
df.head()

## Next

- ตอนนี้คุณมี train/validation/test อยู่บน S3 แล้ว (`processed_*_data_s3_uri`) เพื่อใช้ต่อใน TrainingStep
- Feature Store ingest เสร็จแล้ว และสคริปต์จะรอจน Glue table พร้อม
  - ไปที่ Athena → เลือก database/table ตามที่ log ของ Processing job แสดง (`DataCatalogConfig: <db> <table>`)

ตัวอย่าง query:

```sql
SELECT *
FROM "<database>"."<table>"
LIMIT 10;
```